This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [ ]:
pip install python-dotenv

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import json
import requests
from requests.exceptions import HTTPError

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
try:
    response = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X')
    response.raise_for_status()
    jsonResponse = response.json()

    # I use two different way of indexing:
    # In the first apporach I apply list to the dictionaries:
    response_values = list(jsonResponse.values())
    list_response_items = list(response_values[0].items())

    print("First approach: \n")
    for i in range(len(list_response_items[8][1])):
        print(f"{list_response_items[8][1][i]} = {list_response_items[17][1][0][i]}")
    
    # In the second apporach I use the keys in the dictionaries:
    print("\nSecond approach: \n")
    list_column_names = list(response_values[0]['column_names'])
    list_first_data_values = list(response_values[0]['data'][0])
    for i in range(len(list_column_names)):
        print(f"{list_column_names[i]} = {list_first_data_values[i]}")
    
except HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as exp:
    print(f"Exception occurred: {exp}")

First approach: 

Date = 2020-12-01
Open = 112.2
High = 112.2
Low = 111.5
Close = 112.0
Change = None
Traded Volume = 51.0
Turnover = 5703.0
Last Price of the Day = None
Daily Traded Units = None
Daily Turnover = None

Second approach: 

Date = 2020-12-01
Open = 112.2
High = 112.2
Low = 111.5
Close = 112.0
Change = None
Traded Volume = 51.0
Turnover = 5703.0
Last Price of the Day = None
Daily Traded Units = None
Daily Turnover = None


In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [ ]:
# Tasks #1 & #2

list_column_names = list(response_values[0]['column_names'])
list_data_values = list(response_values[0]['data'])

dict_all_prices = {}
for price_data in list_data_values:
    dict_prices = {}
    if price_data[0][0:4] == '2017':
        for i in range(1, len(price_data)):
            dict_prices[list_column_names[i]] = price_data[i]
        dict_all_prices[price_data[0]] = dict_prices
print (dict_all_prices)

In [ ]:
# Task #3

highest_opening_price = 0
lowest_opening_price = 1000

for prices in dict_all_prices.values():
    if prices['Open'] is not None:
        if highest_opening_price < prices['Open']:
            highest_opening_price = prices['Open']

        if lowest_opening_price > prices['Open']:
            lowest_opening_price = prices['Open']
    
print(f"Highest opening price: {highest_opening_price}")
print(f"Lowest opening price: {lowest_opening_price}")

In [ ]:
# Task #4

largest_price_change = 0

dict_all_prices_values = dict_all_prices.values()
for prices in dict_all_prices_values:
    if prices['High'] is not None and prices['Low'] is not None:
        if largest_price_change < (prices['High']-prices['Low']):
            largest_price_change = prices['High']-prices['Low']
            
print(f"Largest price change in one day: {largest_price_change:.2f}")

In [ ]:
# Task #5
# Note: I assumed that the question is NOT asking for any two *consecutive* days.

largest_price_change = 0

list_all_prices_values = list(dict_all_prices.values())
for i in range(len(list_all_prices_values)):
    for j in range(i+1, len(list_all_prices_values)):
        if list_all_prices_values[i]['Close'] is not None and list_all_prices_values[j]['Close'] is not None:
            if largest_price_change < abs(list_all_prices_values[i]['Close']-list_all_prices_values[j]['Close']):
                largest_price_change = abs(list_all_prices_values[i]['Close']-list_all_prices_values[j]['Close'])
            
print(f"Largest price change between two days: {largest_price_change:.2f}")

In [ ]:
# Task #6

sum_daily_traded_volume = 0
count_days = 0

dict_all_prices_values = dict_all_prices.values()
for prices in dict_all_prices_values:
    if prices['Traded Volume'] is not None:
        sum_daily_traded_volume += prices['Traded Volume']
        count_days += 1

average_daily_traded_volume = sum_daily_traded_volume / count_days

print(f"Average daily traded colume: {average_daily_traded_volume:.2f}")

In [ ]:
# Task #7

list_traded_volume = []

dict_all_prices_values = dict_all_prices.values()
for prices in dict_all_prices_values:
    if prices['Traded Volume'] is not None:
        list_traded_volume.append(prices['Traded Volume'])

list_traded_volume.sort()

print(f"Median daily trading colume: {list_traded_volume[int(len(list_traded_volume)/2)+1]:.2f}")